In [1]:
from openai import OpenAI

In [2]:
client = OpenAI(api_key=)
 
assistant = client.beta.assistants.create(
  name="URA Assistant",
  instructions="You are an expert on the built environment, property sales and urban planning sector. Use the given documents to answer questions relating to these areas. If you are unable to find relevant information or do not know, say that you are unable to help",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [13]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_D9uDZLsSQMgGW8Z6T0JzCOrx', created_at=1716171377, description=None, instructions='You are an expert on the built environment, property sales and urban planning sector. Use the given documents to answer questions relating to these areas. If you are unable to find relevant information or do not know, say that you are unable to help', metadata={}, model='gpt-4o', name='URA Assistant', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_VpbRo3QZvpqM8yhsxtDllf7t'])), top_p=1.0)]


In [58]:
my_updated_assistant = client.beta.assistants.update(
  #ASSISTANT_ID,
  instructions="You are an expert on the built environment, property sales and urban planning sector. Use only the given documents to answer questions relating to these areas. After your summarised first respond, generate the specific chunks you have used. Be very strict in the interpretation of the given information and ensure your response strictly adheres to it.  If you are unable to find relevant information or do not know, say that you are unable to help",
  name="HR Helper",
  tools=[{"type": "file_search"}],
  model="gpt-3.5-turbo"
)

In [26]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="URA_Test")
 
# Ready the files for upload to OpenAI
file_paths = ["data/23q4_stats.html"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [20]:
# Ready the files for upload to OpenAI
file_paths = ["../data/23q4_stats.html"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [52]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [59]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Give me a comparison of the stock and vacancy of residential units as compared to office units for the 4th Quarter 2023 and 1st Quarter 2023. If there are any web links, please provide me with them.",
      # Attach the new file to the message.
    }
  ]
)

In [54]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
client = OpenAI()
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Nick Tang. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > file_search


assistant > In the 4th Quarter of 2023, the stock of completed private residential units (excluding ECs) increased by 3,925 units, while the stock of occupied private residential units (excluding ECs) increased by 4,926 units. This led to a decrease in the vacancy rate of completed private residential units (excluding ECs) to 8.1% at the end of the quarter. In comparison, the stock of office space increased by 2,000 sq m (nett) in the same quarter, and the island-wide vacancy rate of office space decreased to 9.9%[0][1].

For the 1st Quarter of 2023, the exact figures for residential units and office units are not provided in the search results. However, you can access the full details, including the stock and vacancy of residential units and office units for the 1st Quarter of 2023, by referring to the provided document: "23q4_stats.html"[0].
[0] 23q4_stats.html
[1] 23q4_stats.html
[2] 23q4_stats.html


In [60]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Nick Tang. The user has a premium account. Respond with 'I am unable to help you with that' if you are unable to find relevant information",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > file_search


assistant > Here is the comparison of stock and vacancy of residential units and office units for the 4th Quarter 2023:

**Residential Units (4th Quarter 2023):**
- The stock of completed private residential units (excluding ECs) increased by 3,925 units compared to the previous quarter.
- The vacancy rate of completed private residential units decreased to 8.1% at the end of the 4th Quarter 2023[0].

**Office Units (4th Quarter 2023):**
- The stock of office space increased by 2,000 sq m in the 4th Quarter 2023.
- The island-wide vacancy rate of office space decreased to 9.9% at the end of the 4th Quarter 2023[1].

Unfortunately, the document did not contain information about the 1st Quarter 2023 for a direct comparison.
[0] 23q4_stats.html
[1] 23q4_stats.html


In [24]:
vector_store_files = client.beta.vector_stores.files.list(
  vector_store_id=vector_store.id
)
print(vector_store_files)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-DpQsR16Xq1zypA0cTlei132M', created_at=1716172782, last_error=None, object='vector_store.file', status='completed', usage_bytes=84470, vector_store_id='vs_VpbRo3QZvpqM8yhsxtDllf7t'), VectorStoreFile(id='file-H8uTDvLmJOm2vci3NEvwUPqO', created_at=1716171386, last_error=None, object='vector_store.file', status='completed', usage_bytes=21150, vector_store_id='vs_VpbRo3QZvpqM8yhsxtDllf7t')], object='list', first_id='file-DpQsR16Xq1zypA0cTlei132M', last_id='file-H8uTDvLmJOm2vci3NEvwUPqO', has_more=False)


In [25]:
deleted_vector_store = client.beta.vector_stores.delete(
  vector_store_id=vector_store.id
)
print(deleted_vector_store)

VectorStoreDeleted(id='vs_VpbRo3QZvpqM8yhsxtDllf7t', deleted=True, object='vector_store.deleted')
